In [ ]:
pip install torch torchvision pytorch-tabnet xgboost scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from skimage.measure import shannon_entropy
from skimage.filters import sobel, gabor
from scipy import stats

class ImageFeatureExtractor:
    def __init__(self):
        self.hog_params = {
            'orientations': 8,
            'pixels_per_cell': (16, 16),
            'cells_per_block': (2, 2)
        }

    def extract_features(self, image_path):
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Could not read image at {image_path}")

        img = cv2.resize(img, (512, 512))
        _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        features = {}

        # Stroke Morphology
        features['stroke_density'] = np.sum(binary > 0) / (512*512)
        features['contour_count'] = len(contours)

        # Texture Analysis
        hog_feat = hog(img, **self.hog_params)
        features['hog_mean'] = np.mean(hog_feat)
        features['hog_skew'] = stats.skew(hog_feat)

        lbp = local_binary_pattern(img, P=8, R=1)
        features['lbp_entropy'] = shannon_entropy(lbp)

        # Spatial Organization
        if contours:
            contours_combined = np.vstack(contours)
            x, y = contours_combined.squeeze().T
            features['writing_span_x'] = np.ptp(x)
            features['writing_span_y'] = np.ptp(y)
            features['aspect_ratio'] = features['writing_span_y'] / (features['writing_span_x'] + 1e-6)
        else:
            features.update({'writing_span_x': 0, 'writing_span_y': 0, 'aspect_ratio': 0})

        # Dynamic Simulation
        edges = sobel(binary)
        features['edge_density'] = np.sum(edges > 0.1) / (512*512)

        # Advanced Features
        gabor_filt_real, _ = gabor(img, frequency=0.6)
        features['gabor_energy'] = np.mean(gabor_filt_real**2)

        # Entropy Measures
        features['shannon_entropy'] = shannon_entropy(binary)
        features['spatial_entropy'] = self._calc_spatial_entropy(binary)

        # Stroke Regularity
        if len(contours) > 1:
            areas = [cv2.contourArea(c) for c in contours]
            features['stroke_size_cv'] = np.std(areas) / np.mean(areas)
        else:
            features['stroke_size_cv'] = 0

        return features

    def _calc_spatial_entropy(self, image):
        grid = cv2.resize(image, (8, 8))
        return shannon_entropy(grid)


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import joblib
import zipfile
from sklearn.pipeline import Pipeline
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
import xgboost as xgb

# ==== Load models ====
# 1. image model (feature extractor)
#image_model = joblib.load("/content/alzheimer_image_model.pkl")  # sklearn model

# 2. Scaler
scaler = joblib.load("/content/scaler_fold1.pkl")

# 3. XGBoost
xgb_model = joblib.load("/content/xgboost_fold5.pkl")

# 4. Stacker
stacker = joblib.load("/content/stacker_fold5.pkl")

# 5. TabNet (load after extraction)
with zipfile.ZipFile("/content/tabnet_fold5_seed42.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/tabnet_model/")
tabnet = TabNetClassifier()
tabnet.load_model("/content/tabnet_fold5_seed42.zip")

# ==== Image preprocessing ====


SELECTED_FEATURES = [  # 80 features you saved
    'air_time2', 'max_y_extension2', 'paper_time2', 'total_time2', 'mean_jerk_in_air3', 'mean_speed_on_paper3', 'pressure_var3', 'total_time3', 'pressure_mean4', 'pressure_var4', 'air_time5', 'pressure_mean5', 'air_time6', 'pressure_mean6', 'total_time6', 'air_time7', 'gmrt_in_air7', 'mean_gmrt7', 'mean_speed_in_air7', 'total_time7', 'air_time8', 'mean_gmrt8', 'mean_speed_on_paper8', 'paper_time8', 'pressure_mean8', 'total_time8', 'disp_index9', 'mean_jerk_in_air9', 'mean_jerk_on_paper9', 'paper_time9', 'pressure_mean9', 'total_time9', 'gmrt_on_paper10', 'paper_time10', 'paper_time11', 'paper_time12', 'total_time12', 'air_time13', 'total_time13', 'mean_gmrt14', 'air_time15', 'total_time15', 'air_time16', 'total_time16', 'air_time17', 'gmrt_in_air17', 'mean_gmrt17', 'mean_jerk_in_air17', 'mean_speed_in_air17', 'paper_time17', 'total_time17', 'total_time18', 'max_y_extension19', 'num_of_pendown19', 'pressure_mean19', 'pressure_var19', 'disp_index20', 'max_y_extension20', 'paper_time20', 'total_time20', 'max_x_extension21', 'pressure_mean21', 'air_time22', 'disp_index22', 'paper_time22', 'total_time22', 'air_time23', 'disp_index23', 'gmrt_in_air23', 'mean_gmrt23', 'paper_time23', 'total_time23', 'air_time24', 'mean_jerk_on_paper24', 'total_time24', 'disp_index25', 'max_y_extension25', 'mean_gmrt25', 'mean_speed_in_air25', 'paper_time25'
]

IMAGE_FEATURE_MAP = {
    'stroke_density': 0.12,
    'contour_count': 34,
    # Add mappings if any of the selected 80 features are extracted from image
}

def extract_features(image_path):
    extractor = ImageFeatureExtractor()
    image_features = extractor.extract_features(image_path)

    full_feature_vector = []
    for feat in SELECTED_FEATURES:
        # Map image features if available, otherwise zero
        value = image_features.get(feat, 0.0)
        full_feature_vector.append(value)

    return np.array(full_feature_vector).reshape(1, -1)



    # Manually run the pipeline except the final classifier
    if isinstance(image_model, Pipeline):
        steps = list(image_model.named_steps.items())
        for name, step in steps[:-1]:  # exclude the classifier
            flat = step.transform(flat)
        features = flat
    else:
        raise ValueError("image_model is not a Pipeline")

    return features


# ==== Prediction Pipeline ====
def predict_alzheimer(image_path):
    features = extract_features(image_path)
    features_scaled = scaler.transform(features)

    pred_tabnet = tabnet.predict_proba(features_scaled)[:, 1]
    pred_xgb = xgb_model.predict_proba(features_scaled)[:, 1]

    # Combine predictions for stacker
    stacked_input = np.vstack([pred_tabnet, pred_xgb]).T
    final_prediction = stacker.predict(stacked_input)
    final_prob = stacker.predict_proba(stacked_input)[:, 1]

    return final_prediction[0], final_prob[0]

# ==== Example usage ====
image_path = "/content/Screenshot 2025-02-10 132211.png"
label, confidence = predict_alzheimer(image_path)
print(f"Predicted Label: {'Alzheimer' if label == 1 else 'Healthy'}, Confidence: {confidence:.2f}")


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


Predicted Label: Healthy, Confidence: 0.22
